<a href="https://colab.research.google.com/github/YangSooRim/CV2022-Final/blob/main/Original_NetVLAD_base_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/My Drive/컴퓨터비전/NetVLAD')

!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 128 kB/s 


In [3]:
cd /content/drive/My Drive/컴퓨터비전/NetVLAD

/content/drive/My Drive/컴퓨터비전/NetVLAD


In [4]:
ls -al

total 478091
-rw------- 1 root root     29840 Aug  7  2021 berlin.mat
drwx------ 2 root root      4096 May 12 02:25 ckpt/
-rw------- 1 root root     10244 May 12 02:09 .DS_Store
-rw------- 1 root root    114350 Jun 17 08:38 NetVLAD_base_code.ipynb
-rw------- 1 root root  87315062 Sep 26  2021 pittsburgh_checkpoint.pth.tar
drwx------ 2 root root      4096 May 12 02:25 Query_resized/
drwx------ 2 root root      4096 May 12 02:25 Reference_resized/
-rw------- 1 root root 171668187 Jun 17 08:59 resnet101_128.pth
-rw------- 1 root root 171143643 Jun 17 08:53 resnet101_64.pth
-rw------- 1 root root    138913 Jun 17 08:21 submit.json
-rw------- 1 root root  59129550 Jun 17 09:02 vgg16_128.pth


In [5]:
!nvidia-smi

Fri Jun 17 10:37:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import scipy.io as sio

arr = sio.loadmat('berlin.mat')
print('arr')
print(arr)
print('\n\n')

# a = arr['a']
# b = arr['b']

# print('a')
# print(a)

# print('b')
# print(b)

arr
{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Dec  9 12:47:24 2020', '__version__': '1.0', '__globals__': [], 'dbStruct': array([[array(['Reference_resized/Image_(2).jpg    ',
               'Reference_resized/Image_(3).jpg    ',
               'Reference_resized/Image_(4).jpg    ',
               'Reference_resized/Image_(5).jpg    ',
               'Reference_resized/Image_(6).jpg    ',
               'Reference_resized/Image_(7).jpg    ',
               'Reference_resized/Image_(8).jpg    ',
               'Reference_resized/Image_(9).jpg    ',
               'Reference_resized/Image_(10).jpg   ',
               'Reference_resized/Image_(11).jpg   ',
               'Reference_resized/Image_(12).jpg   ',
               'Reference_resized/Image_(13).jpg   ',
               'Reference_resized/Image_(14).jpg   ',
               'Reference_resized/Image_(15).jpg   ',
               'Reference_resized/Image_(16).jpg   ',
               'Reference_resized/Image_

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision

from torchvision.models import resnet18,vgg16, resnet101

torch.manual_seed(777)

In [8]:
class NetVLAD(nn.Module):
    """NetVLAD layer implementation"""

    def __init__(self, num_clusters=12, dim=128, alpha=100.0,normalize_input=True):
        """
        Args:
            num_clusters : int
                The number of clusters
            dim : int
                Dimension of descriptors
            alpha : float
                Parameter of initialization. Larger value is harder assignment.
            normalize_input : bool
                If true, descriptor-wise L2 normalization is applied to input.
        """
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim
        self.alpha = alpha
        self.normalize_input = normalize_input
        self.conv = nn.Conv2d(dim, num_clusters, kernel_size=(1, 1), bias=False)
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim))
        self._init_params()

    def _init_params(self):
        self.conv.weight = nn.Parameter(
            (2.0 * self.alpha * self.centroids).unsqueeze(-1).unsqueeze(-1)
        )

    def forward(self, x):

        N, C = x.shape[:2]

        if self.normalize_input:
            x = F.normalize(x, p=2, dim=1)  # across descriptor dim

        # soft-assignment
        soft_assign = self.conv(x).view(N, self.num_clusters, -1)
        soft_assign = F.softmax(soft_assign, dim=1)

        x_flatten = x.view(N, C, -1)
        
        # calculate residuals to each clusters
        vlad = torch.zeros([N, self.num_clusters, C], dtype=x.dtype, layout=x.layout, device=x.device)
        for C in range(self.num_clusters): # slower than non-looped, but lower memory usage 
            residual = x_flatten.unsqueeze(0).permute(1, 0, 2, 3) - \
                    self.centroids[C:C+1, :].expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0)
            residual *= soft_assign[:,C:C+1,:].unsqueeze(2)
            vlad[:,C:C+1,:] = residual.sum(dim=-1)

        vlad = F.normalize(vlad, p=2, dim=2)  # intra-normalization
        vlad = vlad.view(x.size(0), -1)  # flatten
        vlad = F.normalize(vlad, p=2, dim=1)  # L2 normalize

        return vlad

In [9]:
class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
# VGG 16
encoder = vgg16(pretrained=True)

# ResNet18
# encoder = resnet18(pretrained=True)

# ResNet101
# encoder = resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [13]:
# VGG16

layers = list(encoder.features.children())[:-2]

for l in layers[:-5]: 
    for p in l.parameters():
        p.requires_grad = False

model = nn.Module() 
encoder = nn.Sequential(*layers)

In [ ]:
# ResNet

modules = list(encoder.children())[:-1]      # delete the last fc layer.
encoder = nn.Sequential(*modules)
for param in encoder.parameters():
  param.requires_grad = False

In [14]:

model.add_module('encoder', encoder) #origin
dim = list(encoder.parameters())[-1].shape[0]  # last channels (512)

# Define model for embedding
net_vlad = NetVLAD(num_clusters=16, dim=dim) # edit
model.add_module('pool', net_vlad)

model = model.cuda()

In [ ]:
# Pretrained Model Load(Pittsburgh 30k)
load_model = torch.load('/content/drive/MyDrive/컴퓨터비전/NetVLAD/resnet101_128.pth')
model.load_state_dict(load_model)

In [15]:
# Pretrained Model Load(Pittsburgh 30k)
load_model = torch.load('./pittsburgh_checkpoint.pth.tar')
model.load_state_dict(load_model['state_dict'])

<All keys matched successfully>

In [16]:
import torch.utils.data as data
import torchvision.transforms as transforms

from random import choice
from os.path import join, exists
from collections import namedtuple
from scipy.io import loadmat
from PIL import Image
from sklearn.neighbors import NearestNeighbors

root_dir = '/content/drive/My Drive/컴퓨터비전/NetVLAD'

def parse_dbStruct(path):
    mat = loadmat(path)

    matStruct = mat['dbStruct'][0]

    dataset = 'dataset'

    whichSet = 'VPR'

    dbImage = matStruct[0]
    locDb = matStruct[1]

    qImage = matStruct[2]
    locQ = matStruct[3]

    numDb = matStruct[4].item()
    numQ = matStruct[5].item()

    posDistThr = matStruct[6].item()
    posDistSqThr = matStruct[7].item()

    return dbStruct(whichSet, dataset, dbImage, locDb, qImage, 
            locQ, numDb, numQ, posDistThr, 
            posDistSqThr)
  
dbStruct = namedtuple('dbStruct', ['whichSet', 'dataset',
  'dbImage', 'locDb', 'qImage', 'locQ', 'numDb', 'numQ',
  'posDistThr', 'posDistSqThr'])
  
class BerlinDataset(data.Dataset) :
  
  def __init__(self,condition='train') :
    self.dbStruct = parse_dbStruct('/content/drive/My Drive/컴퓨터비전/NetVLAD/berlin.mat')
    self.input_transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
      ])
    
    self.condition = condition

    if self.condition == 'train' :
      self.images = [join(root_dir, dbIm.replace(' ','')) for dbIm in self.dbStruct.dbImage]
    elif self.condition == 'test' :
      self.images = [join(root_dir, qIm.replace(' ','')) for qIm in self.dbStruct.qImage]
    else :
      self.images = [join(root_dir, dbIm.replace(' ','')) for dbIm in self.dbStruct.dbImage]
    
    self.positives = None
    self.distances = None

    self.getPositives()
  
  def __getitem__(self, index):

      if self.condition == 'train' :
        img = Image.open(self.images[index])
        img = self.input_transform(img)

        pos_list = self.positives[index].tolist()
        pos_list.remove(index)
        pos_index = self.positives[index][np.random.randint(0,len(self.positives[index]))]
        pos_img = Image.open(self.images[pos_index])
        pos_img = self.input_transform(pos_img)

        pos_list = pos_list + [index]
        neg_index = choice([i for i in range(len(self.images)) if i not in pos_list])
        neg_img = Image.open(self.images[neg_index])
        neg_img = self.input_transform(neg_img)

        img = torch.stack([img,pos_img,neg_img],dim=0)
        label = torch.Tensor([0, 0, 1])

        return img, label

      elif self.condition == 'test' :
        img = Image.open(self.images[index])
        img = self.input_transform(img)

        return img
      
      else :
        img = Image.open(self.images[index])
        img = self.input_transform(img)

        return img


  def __len__(self):
      return len(self.images)

  def getPositives(self):
      # positives for evaluation are those within trivial threshold range
      #fit NN to find them, search by radius
      if  self.condition == 'train' :
          knn = NearestNeighbors(n_jobs=1)
          knn.fit(self.dbStruct.locDb)

          self.distances, self.positives = knn.radius_neighbors(self.dbStruct.locDb,radius=self.dbStruct.posDistThr)
      else :
          knn = NearestNeighbors(n_jobs=1)
          knn.fit(self.dbStruct.locDb)

          self.distances, self.positives = knn.radius_neighbors(self.dbStruct.locQ,
                  radius=self.dbStruct.posDistThr)
      
      return self.positives

In [17]:
train_dataset = BerlinDataset(condition="train")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1,shuffle=True,num_workers=0)   
print(train_loader)

In [18]:
epochs = 20
global_batch_size = 8
lr = 0.00001
momentum = 0.9
weightDecay = 0.001
losses = AverageMeter()
best_loss = 100.0
margin = 0.1 

criterion = nn.TripletMarginLoss(margin=margin**0.5, p=2, reduction='sum').cuda()
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum, weight_decay=weightDecay)

model.train()

for epoch in range(epochs):
  for batch_idx, (train_image,train_label) in enumerate(train_loader) :
    output_train = model.encoder(train_image.squeeze().cuda())
    output_train = model.pool(output_train)
    triplet_loss = criterion(output_train[0].reshape(1,-1),output_train[1].reshape(1,-1),output_train[2].reshape(1,-1))

    if batch_idx == 0 :
      optimizer.zero_grad()

    triplet_loss.backward(retain_graph=True)
    losses.update(triplet_loss.item())

    if (batch_idx +1) % global_batch_size == 0 :
      for name, p in model.named_parameters():
        if p.requires_grad:
          p.grad /= global_batch_size
    
        optimizer.step()
        optimizer.zero_grad()

    if batch_idx % 20 == 0 and batch_idx != 0:
      print('epoch : {}, batch_idx  : {}, triplet_loss : {}'.format(epoch,batch_idx,losses.avg))
  
  if best_loss > losses.avg :
    best_save_name = 'best_model.pt'
    best_path = F"./ckpt/{best_save_name}" 
    torch.save(model.state_dict(), best_path)
    
  model_save_name = 'model_{:02d}.pt'.format(epoch)
  path = F"./ckpt/{model_save_name}" 
  torch.save(model.state_dict(), path)

epoch : 0, batch_idx  : 20, triplet_loss : 0.09957774764015562
epoch : 0, batch_idx  : 40, triplet_loss : 0.11885960073005862
epoch : 0, batch_idx  : 60, triplet_loss : 0.12998900257173132
epoch : 0, batch_idx  : 80, triplet_loss : 0.14007304847976307
epoch : 0, batch_idx  : 100, triplet_loss : 0.1340078050547307
epoch : 0, batch_idx  : 120, triplet_loss : 0.13920061992219657
epoch : 0, batch_idx  : 140, triplet_loss : 0.1379302532960337
epoch : 0, batch_idx  : 160, triplet_loss : 0.13918040887169217
epoch : 0, batch_idx  : 180, triplet_loss : 0.14168001670205133
epoch : 0, batch_idx  : 200, triplet_loss : 0.1399022020510773
epoch : 0, batch_idx  : 220, triplet_loss : 0.14020681057580456
epoch : 0, batch_idx  : 240, triplet_loss : 0.14138790136550966
epoch : 0, batch_idx  : 260, triplet_loss : 0.1447170899745605
epoch : 0, batch_idx  : 280, triplet_loss : 0.14568978866224186
epoch : 0, batch_idx  : 300, triplet_loss : 0.1458705430014981
epoch : 1, batch_idx  : 20, triplet_loss : 0.1455

In [19]:
from tqdm import tqdm

cluster_dataset = BerlinDataset(condition="cluster")
cluster_loader = torch.utils.data.DataLoader(cluster_dataset, batch_size=1,shuffle=False,num_workers=0) 

train_feature_list = list()

model.eval()

with torch.no_grad():
  for batch_idx, train_image in tqdm(enumerate(cluster_loader)) :
    output_train = model.encoder(train_image.cuda())
    output_train = model.pool(output_train)
    train_feature_list.append(output_train.squeeze().detach().cpu().numpy())

train_feature_list = np.array(train_feature_list)

314it [00:14, 21.39it/s]


In [20]:
train_feature_list

array([[-0.00558853,  0.00549366, -0.00803546, ...,  0.00093332,
         0.00708362,  0.01618594],
       [-0.01394073,  0.00718206, -0.00831342, ...,  0.01163198,
         0.01252138, -0.0012169 ],
       [-0.01329479, -0.01361928,  0.00497787, ...,  0.01428977,
        -0.00225681, -0.00979786],
       ...,
       [-0.00061193, -0.00481661,  0.00131227, ..., -0.00659715,
         0.01133988,  0.01298243],
       [-0.00187427,  0.01271733, -0.01184988, ..., -0.00730043,
        -0.00165089,  0.01309863],
       [ 0.00290431,  0.00761643, -0.00399074, ..., -0.01877067,
        -0.00650306, -0.00185753]], dtype=float32)

In [21]:
test_dataset = BerlinDataset(condition="test")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False,num_workers=0)   

In [ ]:
test_feature_list = list()

with torch.no_grad():
  for batch_idx, test_image in tqdm(enumerate(test_loader)) :
    output_test = model.encoder(test_image.cuda())
    output_test = model.pool(output_test)
    test_feature_list.append(output_test.squeeze().detach().cpu().numpy())

test_feature_list = np.array(test_feature_list)

42it [00:13,  3.05it/s]

In [ ]:
import faiss

n_values = [1,5,10,20]

faiss_index = faiss.IndexFlatL2(train_feature_list.shape[1])
faiss_index.add(train_feature_list)

_, predictions = faiss_index.search(test_feature_list, max(n_values))





In [ ]:
import json

file_path = "./submit.json"

data = {}
data['Query'] = list()

for i in range(len(predictions)) :
  data_t = [("id",i),("positive",predictions[i].tolist())]
  data_t = dict(data_t)
  data['Query'].append(data_t)
  
with open(file_path, 'w') as outfile:
    json.dump(data, outfile, indent=4)